In [23]:
import matplotlib
matplotlib.use('nbagg')
print(matplotlib.get_backend())
import os
import matplotlib.pyplot as plt
from os import listdir
import random
import numpy as np
import cv2 as cv
import pandas as pd
import open3d as o3d

nbAgg


In [24]:
dataset_location = 'C:\\Users\\sushl\\Desktop\\papers\\datasets\\GustavIIAdolf'
L = os.listdir(dataset_location)
L.sort()

k = np.array([[2393.952166119461, -3.410605131648481e-13, 932.3821770809047], [0, 2398.118540286656, 628.2649953288065], [0, 0, 1]])


In [25]:
def feature_detection(img,detector):
    if detector == "Fast":
        fast = cv.FastFeatureDetector_create()
        fast.setThreshold(10)
        print(fast.getThreshold())
        kp = fast.detect(img,None)
        orb = cv.ORB_create()
        kp, des = orb.compute(img, kp)
        
    if detector == "Orb":
        # Initiate ORB detector
        orb = cv.ORB_create()
        # find the keypoints with ORB
        kp = orb.detect(img,None)
        # compute the descriptors with ORB
        kp, des = orb.compute(img, kp)
    if detector == "Sift":
        sift = cv.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
    # draw only keypoints location,not size and orientation
    #img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
    #plt.imshow(img2), plt.show()
    return kp,des

In [26]:
def Triangulation(P1, P2, pts1, pts2):
    point_cloud = cv.triangulatePoints(P1, P2, pts1, pts2)
    point_cloud = point_cloud / point_cloud[3,:]
    return point_cloud.T

In [27]:
def projection_matrix(k,R,t):
    Rt = np.zeros((3,4))
    Rt[:3,:3] = R
    Rt[:,3] = t.reshape((3))
    P = k@Rt
    return P

In [28]:
def keypoint_matches(kp1,des1,kp2,des2):
    bf = cv.BFMatcher()
    pts1 = cv.KeyPoint_convert(kp1)
    pts2 = cv.KeyPoint_convert(kp2)
    matches = bf.knnMatch(des1,des2,k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)

    src_p = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_p=np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    common_des1 = np.array([ des1[m.queryIdx] for m in good ])
    common_des2 = np.array([ des2[m.trainIdx] for m in good ])
    common_pts1 = np.float32(src_p)
    common_pts2 = np.float32(dst_p)
    return pts1,pts2,common_pts1,common_pts2,common_des1,common_des2

In [29]:
def optical_flow_matches(imgb,imga,kpa):
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 5,
                      criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    kpb, st, err = cv.calcOpticalFlowPyrLK(imga, imgb, kpa, None, **lk_params)
    ptsb = kpb[st == 1]
    ptsa = kpa[st == 1]
    return ptsa.reshape(-1,1,2),ptsb.reshape(-1,1,2),st

In [30]:
def pnp(p3d,pts,k):
    retval, rvec, t, inliers = cv.solvePnPRansac(p3d,pts, k, (0,0,0,0),useExtrinsicGuess = True ,iterationsCount = 100,reprojectionError = 5.0,confidence = 0.750,flags = cv.SOLVEPNP_ITERATIVE)
    R,Jec = cv.Rodrigues(rvec)
    return R,t

In [31]:
def reprojection_error(p3d,P,pts):
    proj_pt = P@p3d.T
    proj_pt = proj_pt.T
    proj_pt= proj_pt / proj_pt[:,2].reshape((-1,1))
    proj_pt = proj_pt[:,:2]
    err = proj_pt - pts.reshape((-1,2))
    err = np.linalg.norm(err)
    err = err/len(pts)
    return err

In [21]:
#list having all data
data = []
i = 0
#for first two frame using essiential matrix to find pointcloud and initial rotation and translation.
img1 = cv.imread(dataset_location +'/'+ L[0],cv.IMREAD_GRAYSCALE)
kp1,des1 = feature_detection(img1,"Fast")
img2 = cv.imread(dataset_location +'/'+ L[1],cv.IMREAD_GRAYSCALE)
kp2,des2 = feature_detection(img2,"Fast") 

pts1,pts2,common_pts1,common_pts2,common_des1,common_des2 = keypoint_matches(kp1,des1,kp2,des2)

F,mask = cv.findFundamentalMat(common_pts1, common_pts2,cv.FM_RANSAC ,0.4,0.9,mask=None)
E = k.T@F@k

retval, R, t, mask = cv.recoverPose(E, common_pts1, common_pts2, k)

P1 = projection_matrix(k,np.eye(3).astype(np.float32),np.zeros((3,1)))
P2 = projection_matrix(k,R,t)

p3d = Triangulation(P1, P2, common_pts1, common_pts2)
print(i,reprojection_error(p3d,P1,common_pts1))
print(i+1,reprojection_error(p3d,P2,common_pts2))
print(p3d.shape,common_pts1.shape,common_pts2.shape)

data.append([p3d,P1,pts1,common_pts1,common_des1,np.eye(3).astype(np.float32),np.zeros((3,1)),reprojection_error(p3d,P1,common_pts1)])
data.append([p3d,P2,pts2,common_pts2,common_des2,R,t,reprojection_error(p3d,P2,common_pts2)])

i = i +2
while(i<3):
    img3 = cv.imread(dataset_location +'/'+ L[i],cv.IMREAD_GRAYSCALE)
    kp3,des3 = feature_detection(img3,"Fast") 
    pts1,pts2,common_pts1,common_pts2,common_des1,common_des2 = keypoint_matches(kp1,des1,kp2,des2)
    F,mask = cv.findFundamentalMat(common_pts1, common_pts2,cv.FM_RANSAC ,0.4,0.9,mask=None)
    E = k.T@F@k
    retval, R12, t12, mask = cv.recoverPose(E, common_pts1, common_pts2, k)
    R = R12@rotations[-1]
    t = translations[-1] + rotations[-1]@t12
    P1 = projections[-1]
    
    P2 = projection_matrix(k,R,t)
    #print(P1,P2)
    #print(R,t)
    p3d = Triangulation(P1, P2, common_pts1, common_pts2)
    #print(common_pts1.shape)
    #print(p3d.shape)
    #print(reprojection_error(p3d.T,P2,common_pts2))
    #print(reprojection_error(p3d.T,P1,common_pts1))
    #R,t = refine_RT(p3d[:,:3],common_pts2,k)
    #print(R,t)
    #P2 = projection_matrix(k,R,t)
    print(reprojection_error(p3d.T,P2,common_pts2))
    error.append(reprojection_error(p3d.T,P2,common_pts2))
    if error[-1]<5:
        pointclouds.append(Triangulation(P1, P2, common_pts1, common_pts2))    
    rotations.append(R)
    translations.append(t)
    projections.append(P2)
    img1 = img2
    kp1,des1 = kp2,des2
    
    i = i +1

10
10
0 0.9523013012239769
1 1.021118326350893
(4736, 4) (4736, 1, 2) (4736, 1, 2)
1 1.021118326350893 1.026153118294907


In [ ]:
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.subplot(1, 1, 1, projection='3d').plot(p3d[:, 0], p3d[:, 1], p3d[:, 2])
plt.show()

In [ ]:
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'
    # cleaning point cloud
    mean = np.mean(xyz_points[:, :3], axis=0)
    temp = xyz_points[:, :3] - mean
    dist = np.sqrt(temp[:, 0] ** 2 + temp[:, 1] ** 2 + temp[:, 2] ** 2)
    #print(dist.shape, np.mean(dist))
    indx = np.where(dist < np.mean(dist)+ 200)
    xyz_points = xyz_points[indx]
    rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tobytes(),rgb_points[i,1].tobytes(),
                                        rgb_points[i,2].tobytes())))
    fid.close()

In [ ]:
import struct
write_pointcloud("pt.ply",p3d[:,:3])